# Otsitavate lemmade ja alamlemmade nimekirja moodustamine 

[![License: BSD-2-Clause](https://img.shields.io/badge/License-BSD--2--Clause-lightgrey.svg)](https://opensource.org/license/bsd-2-clause/)


Kõik tekstides olevatad sõnavormid pole otsimisel vajalikud.
* Mõned neist on ebaolulised struktuurielemendid nagu sektsiooni numbrid.
* Mõned neist on põhjustatud töötlemisvigadest eelmistes etappides.
* Mõned sõnavormid on trükivigased või lühendid originaaltekstis, mille otsitavaks muutmiseks on vaja luua erinevaid sõnastikke.

Järgnev analüüs adresseerib kõiki neid aspekte. 

In [2]:
import re
import requests

from pandas import DataFrame 
from pandas import read_csv
from pandas import concat
from pandas import merge

from tqdm.auto import tqdm
tqdm.pandas()

## I. Puhastatud tekstide laadimine

In [50]:
sources = {}
sources['state_laws'] = read_csv('../results/cleaned_texts/state_laws.csv', header=0)
sources['government_regulations'] = read_csv('../results/cleaned_texts/government_regulations.csv', header=0)
sources['local_government_acts'] = read_csv('../results/cleaned_texts/local_government_acts.csv', header=0)
sources['government_orders'] = read_csv('../results/cleaned_texts/government_orders.csv', header=0)

## II. Indekseerimisteenuse kasutamine sõnade algvormi leidmiseks

Tekstide edasisel analüüsimisel saame kasutada veebiteennust, mis leiab igale sõnale vastava algvormi.
Kuna mõnele sõnavormile vastab mitu erinevat algvormi (`sadama` -> `sadam`, `sadama`) siis on oluline valida, kas me kasutame kõiki vorme või ühestamist, mille käigus jäetakse alles vaid konteksti sobivad algvormid. 
Erinevalt tekstide puhastamisest kasutame siin dokumentide indekseerimiseks mõeldud veebiteenust.

In [147]:
NUMBER = '[0-9]'
SUPERSCRIPT_SYMBOLS = '[⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻]'
ESTONIAN_LETTER = '[a-z|öäõüžš]'
FOREIGN_LETTER = '[ôíëаa]'

CRE_SUPERCRTIPT_ENDING = re.compile(f'(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}){SUPERSCRIPT_SYMBOLS}$') 
CRE_PSEUDO_SUPERCRTIPT_ENDING = re.compile(f'(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}){{4,}}{NUMBER}$', re.IGNORECASE) 

def index_document(text: str, disambiguation: bool = False):
    """
    Uses web service to extract words and sub-words form document captions

    Returns a four column table with columns lemma, is_sublemma, total_weight
    where the total weight is the total count of the lemma in that role.
    The total count can be fractional as one wordform can have several lemmas 
    and the web service assigns a preference for each of them as weight.
    The sum of weights is one for the lemmas of the wordform and the higher
    weight corresponds to higher likelihood that the lemma is correct.

    If the disambiguation is set then the morphological analysis uses disambiguation
    to elliminate lemma candidates that are unlikely in the contex of the wordform.
    """
    ANALYZER_QUERY = "https://smart-search.tartunlp.ai/api/advanced_indexing/document"
    HEADERS = {"Content-Type": "application/json"}
    POST_DATA_TEMPLATE = {"sources": {"DOC_1":{"content": text}}}
    
    response = requests.post(ANALYZER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    response = response.json()
    tbl = DataFrame(response['tabelid']['indeks_lemmad'], columns=['lemma', 'doc_id', 'start', 'end', 'is_sublemma'])

    # Corrections in tokenisation
    tbl['lemma'] = tbl['lemma'].str.strip()
    
    # Quick hack to eliminate the superscript error
    idx = tbl['lemma'].str.contains(CRE_SUPERCRTIPT_ENDING)
    tbl.loc[idx, 'lemma'] = tbl.loc[idx, 'lemma'].str[:-1]

    # Quick hack to eliminate the superscript error with CO2 exception
    idx = tbl['lemma'].str.contains(CRE_PSEUDO_SUPERCRTIPT_ENDING)
    tbl.loc[idx, 'lemma'] = tbl.loc[idx, 'lemma'].str[:-1]

    output_columns = ['lemma', 'is_sublemma', 'start', 'end', 'weight']

    # Quick hack to find the number of analyses
    analysis_count = (tbl[~tbl['is_sublemma']]
                      .groupby(['start', 'end', 'is_sublemma'], as_index=False)
                      .aggregate(analysis_count=('lemma', len)))

    # Is somewhat incorrect for subword analysis as a single subword can have several sublemmas
    # This causes an issue where subwords get too large weight. 
    return (merge(
                tbl.groupby(['start', 'end', 'lemma'])
                .aggregate(count=('lemma', len))
                .reset_index('lemma'),
                analysis_count, on=['start', 'end']
            ).assign(weight=lambda df: 1/df['analysis_count'])[output_columns]
            .groupby(['lemma', 'is_sublemma'], as_index=False)
            .aggregate(total_weight=('weight','sum'))
            .sort_values(['lemma', 'is_sublemma']))
  

In [148]:
assert not any(index_document('1992 . aasta riigieelarveseadus')['lemma'].str.contains('\s')), 'Ootamatud tühikud'

text = 'Korruptsiooni kriminaalõigusliku reguleerimise konventsiooni ratifitseerimise seadus'
assert not any(index_document(text)['lemma'].str.contains('=', regex=False)), 'ootamatu sümbol ='

text = 'Parlamentaarsest ja valitsustevahelisest koostööst'
assert not any(index_document(text)['lemma'].str.contains('+', regex=False)), 'ootamatu sümbol +'

text = 'Seadus õunte kohta¹' 
assert not any(index_document(text)['lemma'].str.contains(CRE_SUPERCRTIPT_ENDING)), 'Subskript jääb eraldamata'

text = 'Seadus CO2 ja õunte kohta1'
assert not any(index_document(text)['lemma'].str.contains(CRE_PSEUDO_SUPERCRTIPT_ENDING)), 'Subskript jääb eraldamata'

In [150]:
# Note that jaht and jahtima should ideally get weight 0.5 but they get weight 1.0
display(index_document('Daam sülekoeraga ja mees jahikoeraga.').style.set_caption('Daam sülekoeraga ja mees jahikoeraga.'))
display(index_document('Punameremao. Sadama.').style.set_caption('Punameremao jalg. Sadama.'))

,lemma,is_sublemma,total_weight
0,daam,False,1.000000
1,jahikoer,False,1.000000
2,jaht,False,1.000000
3,jahtima,False,1.000000
4,koer,False,2.000000
5,mees,False,0.500000
6,mesi,False,0.500000
7,sülekoer,False,1.000000
8,süli,False,1.000000


,lemma,is_sublemma,total_weight
0,madu,False,0.500000
1,magu,False,0.500000
2,meremadu,False,0.500000
3,meremagu,False,0.500000
4,meri,False,0.500000
5,puna,False,0.500000
6,punama,False,0.500000
7,punameremadu,False,0.500000
8,punameremagu,False,0.500000
9,punameri,False,0.500000


## III. Lemmavormide täiendav puhastamine ja pealkirjade analüüs

TODO: See jutt peaks olema irrelevantne

Kuna tekstide puhastamisetapis jätsime teadlikult sisse mõned vead:

* superskripti sümbol on lisatud nimisõnale,
* tekstile järgneb sidekriipsuga number,
* valesti kirjutatud sõnad tüüpi 17-aastane,

siis nüüd oleks õige neid korrektselt käsitleda. Antud vigadest on vaja vaid parandada need, kus superskripti sümbolile järgneb number ja tekstile järgneb sidekriipsuga number. Teistel juhtudel saab indekseerija osalemmade leidmisega korrektselt hakkama.
Lihtsuse mõttes teeme asenduse peale esialgset indekseerimist. Absoluutselt korrektne lahendus oleks vastavad vead ära parandada eelmises faasis kasutades parandussõnasikku `vigane sõnavorm -> asendatav text`.

In [151]:
all_documents = concat([source for _,source in sources.items()], axis=0)

result = [None] *  len(all_documents)
for i, caption in tqdm(enumerate(all_documents['document_title']), total=len(all_documents)):
    result[i] = index_document(caption).assign(doc_id = i)

    if i > 1000:
        break

result = concat(result, axis=0).reset_index(drop=True)
display(result)

  0%|          | 0/58837 [00:00<?, ?it/s]

,lemma,is_sublemma,total_weight,doc_id
0,ajutine,False,1.0,0
1,konventsioon,False,1.0,0
2,seaduma,False,0.5,0
3,seadus,False,0.5,0
4,siss,False,1.0,0
...,...,...,...,...
12590,piir,False,1.0,1001
12591,riigipiir,False,1.0,1001
12592,riik,False,1.0,1001
12593,seaduma,False,0.5,1001


### Lemmade validatsioon

In [168]:
NUMBER_SYMBOLS = '[0-9]'
SUPERSCRIPT_SYMBOLS = '[⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻]'
ESTONIAN_LETTER = '[a-z|öäõüžš]'
FOREIGN_LETTER = '[ôíëаa]'

CRE_SUPERCRTIPT_ENDING = re.compile(f'(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}){SUPERSCRIPT_SYMBOLS}$') 
CRE_PSEUDO_SUPERCRTIPT_ENDING = re.compile(f'(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}){{4,}}{NUMBER_SYMBOLS}$', re.IGNORECASE) 

In [169]:
assert not any(result['lemma'].str.contains('\s')), 'Ootamatud tühikud'

idx = result['lemma'].str.contains(CRE_SUPERCRTIPT_ENDING)
display(result[idx].style.set_caption('Võimalikud eraldamata supersriptid'))

idx = result['lemma'].str.contains(CRE_PSEUDO_SUPERCRTIPT_ENDING)
display(result[idx].style.set_caption('Võimalikud eraldamata pseudosupersriptid'))

,lemma,is_sublemma,total_weight,doc_id


,lemma,is_sublemma,total_weight,doc_id


### Kureeritud järjelparandused pärisnimedele


In [164]:
# Use correction map for proper names
proper_names = read_csv('inputs/proper_names.csv')
idx = result['lemma'].isin(proper_names['lemma'])
result.loc[idx, 'lemma'] = (merge(result.loc[idx, 'lemma'].reset_index(), proper_names, on='lemma', how='left')
                            .set_index('index')['korrektne_lemma'])

In [8]:
# old_result = result.copy()

In [536]:
# result = old_result.copy()

### Lemmade sagedustabeli loomine

In [165]:
lemma_counts = (result.groupby(['lemma', 'is_sublemma'])
                .agg(total_weight=('total_weight', 'sum'), document_count = ('doc_id', lambda x: len(set(x)))))
lemma_counts = lemma_counts.sort_values(['total_weight', 'document_count'], ascending=False).reset_index()
display(lemma_counts)

,lemma,is_sublemma,total_weight,document_count
0,seadus,False,582.50,936
1,seaduma,False,577.50,936
2,riik,False,484.00,251
3,vaba,False,388.00,198
4,vabariik,False,368.00,198
...,...,...,...,...
2077,Beltides,False,0.25,1
2078,Rooma,False,0.25,1
2079,room,False,0.25,1
2080,rooma,False,0.25,1


## IV. Lemmade tabeli valideerimine ja filtreerimine

Lihtsuse mõttes jätame otsitavateks lemmadeks vaid need lemmad, mis ei ole numbrid ega tähelühendid, aga võivad sisaldada erisümbolitena sidekriipse ja apostroofe.

### Lühendite ja arvude eemaldamine

Tehniliselt on lühendite nimiekirja kõige lihtsam hoida eraldi tekstifailis `abbrevations.csv` ning vajadusel seda täiendada.

In [170]:
abbrevations = read_csv('inputs/abbrevations.csv')
idx = lemma_counts['lemma'].isin(abbrevations['lühend'])
display(lemma_counts[idx].sample(10))
selected = ~idx

idx = lemma_counts['lemma'].str.contains(f'^(?:{NUMBER_SYMBOLS}|\s)+$')
display(lemma_counts[idx & selected])
selected &= ~idx

idx = lemma_counts['lemma'].str.len() == 1
display(lemma_counts[idx & selected].sample(10))
selected &= ~ idx

ValueError: Cannot take a larger sample than population when 'replace=False'

### Kuupäevade ja keerukamate arvudega seotud lühendite eemaldamine 


In [171]:
ESTONIAN_LETTER = '[a-z|öäõüžš]'
FOREIGN_LETTER = '[ôíëаa]'
NUMBER_SYMBOLS = '[0-9]'

# Tühikutega eraldatavad sümbolid 
PUNCTUATION_MARK = '[\\.,:;!?¿\\(\\)«»„““ˮ"‟”]'
SPECIAL_SYMBOLS = '[§/%\^]'
SUPERSCRIPT_SYMBOLS = '[⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻]'
SUBSCRIPT_SYMBOLS = '[₀₁₂₃₄₅₆₇₈₉₊₋]'

# Tühikutega asendatavad sümbolid  
WHITESPACE_SYMBOLS = '[\u200e\ufeff]'

# Teised lubatud sümbolid
DASH_SYMBOLS = '(?:‑|-|−|‒|-)'
OTHER_SYMBOLS = '[§/−a\^]'

# Type 01 . 09 . 2017
idx = lemma_counts['lemma'].str.contains(f'^(?:{NUMBER_SYMBOLS}|,|\.|\s|%)+$')
display(lemma_counts[idx & selected])
selected &= ~idx

# Type 10а
idx = lemma_counts['lemma'].str.contains(f'^(?:{NUMBER_SYMBOLS})+(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER})$', case=False) 
display(lemma_counts[idx & selected])
selected &= ~idx

# Type 813-k
idx = lemma_counts['lemma'].str.contains(f'^(?:{NUMBER_SYMBOLS})+{DASH_SYMBOLS}(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER})$', case=False) 
display(lemma_counts[idx & selected])
selected &= ~idx

NameError: name 'selected' is not defined

Allesjäänud numbreid sisaldavad sõnu ei saa algoritmiliselt paremini töödelda. 
Seetõttu moodustame eraldi lubatud sõnade loendi ja jätame alles vaid need numbreid sisaldavad lemmad, mis on kureeritud failis `allowed_names.csv`.

In [172]:
allowed_names = read_csv('inputs/allowed_names.csv')

idx = lemma_counts['lemma'].str.contains('[0-9]')
display(lemma_counts[idx & selected].sample(10))
idx &= ~ lemma_counts['lemma'].isin(allowed_names['lemma'])
lemma_counts.loc[idx & selected, 'lemma'].to_csv('../results/candidate_lists/allowed_names.csv', index=False)

selected &= ~idx

NameError: name 'selected' is not defined

### Allesjäänud lühendite sõnaraamatu koostamine

Lemmade hulka alles jäänud lühenditest on kasulik luua loend faili `abbrevation_dictionary.csv`. See võimaldab meil hiljem tekitada sõnastiku, mille abil pikk otsisõna laiendatakse ka vastavaks lühendiks näiteks Euroopa Liit → EL ja aksiaselts → AS. 

In [543]:
idx = lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER.upper()}|{FOREIGN_LETTER.upper()}|{NUMBER_SYMBOLS})+$')
display(lemma_counts.loc[idx & selected].sample(10))

(lemma_counts.loc[idx & selected, ['lemma']]
 .rename(columns={'lemma': 'lühend'})
 .assign(täisnimi='')
 .sort_values(['lühend'])
 .to_csv('../results/candidate_lists/abbrevation_dictionary.csv', index=False))

,lemma,is_sublemma,total_weight,document_count
9255,LEADER,False,3.0,3
15251,GG,False,1.0,1
15162,CEPT,False,1.0,1
2852,NSV,False,25.0,23
15260,HEV,False,1.0,1
15142,BBMRI,False,1.0,1
15738,WTO,False,1.0,1
15209,EV,False,1.0,1
15254,GSM,False,1.0,1
7055,DNA,False,5.0,5


In [326]:
any(lemma_counts['lemma'] == 'CO2')

False

### Pärisnimede ja initsiaalide korrektne töötlemine

Pärisnimede korral ei toimi lemmatiseerimine alati korrektselt ja seega tuleb need käsitsi läbi vaadata. Absoluutselt korrektne viis pärisnimede tuvastamiseks on nimeolemituvastus. Näiteks [TartuNLP nimeolemituvastus](https://ner.tartunlp.ai) või [EstNLTK nimeolemituvastus](https://github.com/estnltk/estnltk/blob/main/tutorials/nlp_pipeline/D_information_extraction/02_named_entities.ipynb). Antud kontekstis on kõige lihtsam otsida vaid initsiaalidega pärisnimesid, sest nende analüüs on kõige suurem probleem. Nagu eelnevalt teeme vastava manuaalselt kureeritud faili `proper_names.csv`.

In [544]:
idx = lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER})\s\.', case=False)
idx &= ~lemma_counts['is_sublemma']
display(lemma_counts[idx & selected])

(lemma_counts
 .loc[idx & selected & ~lemma_counts['lemma'].isin(proper_names['korrektne_lemma']), ['lemma']]
 .assign(korrektne_lemma=lambda df: df['lemma'].str.title())
 .to_csv('../results/candidate_lists/proper_names.csv', index=False, quoting=csv.QUOTE_ALL))

,lemma,is_sublemma,total_weight,document_count
3121,F . J . Wiedemann,False,21.0,21
5897,J . V . Veski,False,7.0,7
7933,C . R . Jakobson,False,4.0,4
7958,M . Lüdig,False,4.0,4
9195,A . H . Tammsaare,False,3.0,3
11227,O . Luts,False,2.0,2
11251,S . Miloševic,False,2.0,2
15101,A . Hahn,False,1.0,1
15189,E . Vilde,False,1.0,1
15250,G . Ots,False,1.0,1


Eemaldame initsiaalid otsitavate alamsõnade hulgast. Seda saaks paremini teha, aga see on piisavlt täpne muster.

In [545]:
idx = lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}|\s|\.)+$', case = False)
idx &= lemma_counts['lemma'].str.contains('\s')
idx &= lemma_counts['is_sublemma']
display(lemma_counts[idx])
selected &= ~ idx

,lemma,is_sublemma,total_weight,document_count
3120,F . J .,True,21.0,21
5896,J . V .,True,7.0,7
7932,C . R .,True,4.0,4
9194,A . H .,True,3.0,3
9196,A . H . tamm,True,3.0,3
9223,Fr . R .,True,3.0,3
11226,O .,True,2.0,2
15188,E .,True,1.0,1
15249,G .,True,1.0,1


In [546]:
lemma_counts[lemma_counts['lemma'].str.contains('\s') &selected]

,lemma,is_sublemma,total_weight,document_count
3121,F . J . Wiedemann,False,21.0,21
5897,J . V . Veski,False,7.0,7
7933,C . R . Jakobson,False,4.0,4
7958,M . Lüdig,False,4.0,4
9195,A . H . Tammsaare,False,3.0,3
9224,Fr . R . Kreutzwald,False,3.0,3
11227,O . Luts,False,2.0,2
11251,S . Miloševic,False,2.0,2
15101,A . Hahn,False,1.0,1
15189,E . Vilde,False,1.0,1


**Kommentaar:** Tulemus näitab, et esialgne sõnestus sisaldab vigu veebiaadresside osas. 
Meie korrektsioonid enne sõnestamist löövad veebiaadressi kujule xyz . abc . com, mis tekitab väga palju lühikesi sõnu.
Seda on võimalik parandada täpsema eeltöötlusega. Samamoodi on lahti löödud lühendid nagu P.L.C. ja U.K.

### Sidekriipsuga sõnade analüüs

Liitsõnad sisaldavad ainult sidekriipsu mitte teisi sidekriipsulaadseid sümboleid.  
Vaatlus näitab, et peale nimede lemmatiseerimis- ja suurtähevigade muid kahtlasi olukordi pole.  

In [547]:
idx = lemma_counts['lemma'].str.contains(DASH_SYMBOLS)
idx &= ~lemma_counts['lemma'].str.contains('-')
display(lemma_counts[selected & idx])

,lemma,is_sublemma,total_weight,document_count


In [548]:
idx = lemma_counts['lemma'].str.contains(DASH_SYMBOLS)
idx &= lemma_counts['lemma'].map(lambda x: x[0].isupper())
display(lemma_counts[selected & idx])

,lemma,is_sublemma,total_weight,document_count
356,Kohtla-Järve,False,353.000000,349
582,Narva-Jõesuu,False,210.000000,210
583,Narva-Jõgi,True,210.000000,210
637,Väike-Maarja,False,190.000000,189
760,Lääne-Nigula,False,154.000000,152
...,...,...,...,...
22799,Mullutu-Lood,False,0.333333,1
22800,Mullutu-Loode,False,0.333333,1
22846,Vääna-Vit,False,0.333333,1
22847,Vääna-Viti,False,0.333333,1


In [549]:
idx = lemma_counts['lemma'].str.contains(DASH_SYMBOLS)
idx &= ~lemma_counts['lemma'].map(lambda x: x[0].isupper())
display(lemma_counts[selected & idx].sample(10))

,lemma,is_sublemma,total_weight,document_count
1665,sise-esi,True,55.0,55
7277,kogu-külakeskus,True,5.0,5
18287,lugemis-teave,True,1.0,1
17311,keskkonna-infrastruktuur,False,1.0,1
9703,kogu-Küla,True,3.0,3
17420,kivi-Pada,True,1.0,1
8880,ordoviitsiumi-kambrium,False,4.0,3
10771,neto-omama,True,3.0,1
5359,raudtee-ette,True,9.0,7
18030,lasteaed-raamatukogu-külakeskus,False,1.0,1


Probleemsed kohad
* aia-Põhi	
* kivi-Padakõrb	
* aeda-Algkool	

### Ootamatuid sümboleid sisaldavate sõnade analüüs

In [550]:
idx = ~lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}|{DASH_SYMBOLS}|{NUMBER_SYMBOLS})+$', case=False)
idx &= ~lemma_counts['lemma'].isin(proper_names['korrektne_lemma']) 
display(lemma_counts[idx & selected])

,lemma,is_sublemma,total_weight,document_count
16237,eesti . ee,False,1.0,1


### Trükivigadele vastavad lemmad

Nii imelik kui see ka pole, siis Riigi Teataja pealkirjade hulgas esineb ka trükivigu. Neid on kõige lihtsam tuvastada selle järgi, et need esinevad harva ja need pole eestikeelsete sõnade nimistus. Nagu eelnevalt teeme vastava manuaalselt kureeritud faili `misspellings.csv`.

In [551]:
rare_words = lemma_counts[selected & (lemma_counts['total_weight'] < 3) & (lemma_counts['document_count'] <= 3)]

In [552]:
def is_known_word(text: str):
    """
    Uses web service to validate if the word is inside Vabamorf dictionary
    """

    ANALYZER_QUERY = "https://smart-search.tartunlp.ai/api/analyser/process"
    HEADERS = {"Content-Type": "application/json; charset=utf-8"}
    POST_DATA_TEMPLATE = {'params': {}, 'content': text} 

    response = requests.post(ANALYZER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"
    response = response.json()
    return  'mrf' in response['annotations']['tokens'][0]['features']
 
print(is_known_word('kala'))
print(is_known_word('regisri'))

True
False


In [553]:
idx = rare_words['lemma'].progress_map(is_known_word)
name_candidates = rare_words['lemma'].str.contains(f'^{ESTONIAN_LETTER.upper()}|{FOREIGN_LETTER.upper()}')
display(rare_words[~idx & ~name_candidates])

(rare_words
 .loc[~idx & ~name_candidates, ['lemma']]
 .assign(korrektne_lemma=lambda df: df['lemma'])
 .sort_values(by=['lemma'])
 .to_csv('../results/candidate_lists/misspellings.csv', index=False))

  0%|          | 0/11669 [00:00<?, ?it/s]

,lemma,is_sublemma,total_weight,document_count
11311,administratiiv,True,2.000000,2
11318,agraar,True,2.000000,2
11320,agro,True,2.000000,2
11321,aianduser,True,2.000000,2
11322,aianduseri,True,2.000000,2
...,...,...,...,...
23122,regisri,False,0.166667,1
23123,regisrine,False,0.166667,1
23124,regisris,False,0.166667,1
23125,regisrisne,False,0.166667,1


## V. Indeksisse minevate lemmade eksportimine

In [554]:
VALID_SYMBOL = f'{ESTONIAN_LETTER}|{FOREIGN_LETTER}|{DASH_SYMBOLS}|{NUMBER_SYMBOLS}| |\.'
idx = lemma_counts['lemma'].str.contains(f'^(?:{VALID_SYMBOL})+$', case=False)
assert not any(selected & ~idx), "Ootamatus sümbolid valitud sõnade hulgas"

outcome = lemma_counts[idx & selected]
print(f'Täislemmade arv:      {sum(~outcome["is_sublemma"])}')
print(f'Alamsõnavoormide arv: {sum(outcome["is_sublemma"])}')


Täislemmade arv:      15509
Alamsõnavoormide arv: 7148


In [555]:
outcome = (outcome
           .rename(columns={'total_weight': 'occurence_count'})
           .assign(occurence_count=lambda df: df['occurence_count'].round(decimals=1))
           .sort_values(['occurence_count', 'document_count'], ascending=False)
           [['is_sublemma', 'lemma', 'occurence_count', 'document_count']])

outcome.to_csv('../results/word_indices/lemma_index.csv', header=True, index=False)
display(outcome)

,is_sublemma,lemma,occurence_count,document_count
0,False,kord,14730.0,14452
1,True,esi,9856.0,9408
2,True,põhi,8181.0,7831
3,True,määrus,7173.0,7166
4,False,põhimäärus,6949.0,6942
...,...,...,...,...
23122,False,regisri,0.2,1
23123,False,regisrine,0.2,1
23124,False,regisris,0.2,1
23125,False,regisrisne,0.2,1
